In [1]:
import glob

import networkx as nx
import matplotlib.pyplot as plt

In [2]:
# Define flex values and topologies
flex_values = [2, 3, 4]
topologies = ["closed", "linked"]

# Prepare a dictionary to hold filepaths by (flex, topology)
data_dict = {}

for flex in flex_values:
    for topo in topologies:
        pattern = f"./../FJSSP_data/*_{topo}_flex{flex}.txt"
        files = glob.glob(pattern)
        data_dict[(flex, topo)] = files

In [3]:
def validate_data(data: str, target_flex: int, target_topo: str):
    """
    Validate the generated data fit the target flexibility and topology.

    Args:
        data: str, the generated data
        target_flex: int, the target flexibility
        target_topo: str, the target topology
    Returns:
        bool, True if the data fit the target flexibility and topology, False otherwise
    """
    valid = True

    # Parse the first line of data
    number_seq = [int(n) for n in data.split()]
    data_job_cnt = number_seq[0]
    data_machine_cnt = number_seq[1]
    data_flexibility = number_seq[2]

    # Check for topology
    machine_nodes = [f"M{m}" for m in range(data_machine_cnt)]
    MG = nx.Graph()
    MG.add_nodes_from(machine_nodes)

    cur_idx = 3
    for job_idx in range(data_job_cnt):
        # operation 的數量
        operation_cnt = number_seq[cur_idx]
        cur_idx += 1
        for op_idx in range(operation_cnt):
            # 該 operation 可以給幾個機器弄的數量
            op_mode_cnt = number_seq[cur_idx]
            cur_idx += 1
            ava_machine_set = set()
            for mode_idx in range(op_mode_cnt):
                machine_id = number_seq[cur_idx]
                cur_idx += 1
                ava_machine_set.add(machine_id)
                if machine_id > data_machine_cnt:
                    print(f"Error: machine_id {machine_id} > data_machine_cnt {data_machine_cnt}")
                    valid = False

                processing_time = number_seq[cur_idx]
                cur_idx += 1
            if len(ava_machine_set) != data_flexibility:
                print(
                    f"Error: operation {op_idx} of job {job_idx} has {len(ava_machine_set)} available machines, but data_flexibility is {data_flexibility}"
                )
                valid = False

            # C^_2: For each operation, create a clique among all its feasible machines (i.e., fully connect them).
            ava_machine_list = list(ava_machine_set)
            for i in range(len(ava_machine_list)):
                for j in range(i + 1, len(ava_machine_list)):
                    MG.add_edge(f"M{ava_machine_list[i]}", f"M{ava_machine_list[j]}")
    if target_topo == "closed" and nx.number_connected_components(MG) <= 1:
        print(
            f"Error: operation {op_idx} of job {job_idx} has only {nx.number_connected_components(MG)} connected components, but target_topo is closed"
        )
        valid = False
    if target_topo == "linked" and nx.number_connected_components(MG) != 1:
        print(
            f"Error: operation {op_idx} of job {job_idx} has {nx.number_connected_components(MG)} connected components, but target_topo is linked"
        )
        valid = False
    return valid

In [4]:
for flex in flex_values:
    for topo in topologies:
        for f in data_dict[(flex, topo)]:
            with open(f, "r") as file:
                data = file.read()
                valid = validate_data(data, flex, topo)
                if not valid:
                    print(f"❌ {f} is invalid")
                    break
                else:
                    print(f"✅ {f} is valid")

✅ ./../FJSSP_data\ta01_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta02_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta03_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta04_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta05_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta06_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta07_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta08_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta09_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta10_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta11_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta12_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta13_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta14_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta15_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta16_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta17_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta18_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta19_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta20_closed_flex2.txt is valid
✅ ./../FJSSP_data\ta